<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Открытие-и-изучение-файла" data-toc-modified-id="Открытие-и-изучение-файла-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Открытие и изучение файла</a></span></li><li><span><a href="#Разбивка-данных-на-выборки" data-toc-modified-id="Разбивка-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбейте данные на выборки</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Проверка-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-модели-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка модели на тестовой выборке</a></span></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span></li></ul></div>

# Рекомендация тарифов

Требуется модель для задачи классификации, которая будет рекомендовать клиенту подходящий тариф. 

Необходимо построить модель с максимально большим значением *accuracy*. Пороговое значение метрики - не менее 0.75.

## Открытие и изучение файла

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

Сохраним файл в переменную df.

In [2]:
df = pd.read_csv('users_behavior.csv')

Изучим первые 5 строк таблицы.

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Изучим основную информацию о таблице.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Посчитаем доли пользователей с тарифами smart и ultra.

In [5]:
len(df.query('is_ultra == 0')) / len(df) # доля пользователей с тарифом smart

0.693528313627878

In [6]:
len(df.query('is_ultra == 1')) / len(df) # доля пользователей с тарифом ultra

0.30647168637212197

В нашей выборке почти 70% пользователей используют тариф smart. Соответственно, если мы для рекомендации тарифов будем использовать модель, которая по умолчанию предлагает пользователю наиболее часто встречающийся тариф, то accuracy такой модели будет равно 0.69.

## Разбивка данных на выборки

Разделим датасет на признаки и целевой признак. Признаки сохраним в переменную **features**, а целевой признак - в переменную **target**.

In [7]:
features = df.drop(['is_ultra'], axis=1)

In [8]:
target = df['is_ultra']

Теперь разделим признаки и целевой признак на три выбоки: обучающую (train), валидационную (valid) и тестовую (test).

In [9]:
features_new, features_test, target_new, target_test = (
    train_test_split(features, target, test_size=0.2, random_state=12345, stratify=target)
)

In [10]:
features_train, features_valid, target_train, target_valid = (
    train_test_split(features_new, target_new, test_size=0.25, random_state=12345, stratify=target_new)
)

In [11]:
display(features_train.shape)
display(target_train.shape)
display(features_valid.shape)
display(target_valid.shape)
display(features_test.shape)
display(target_test.shape)

(1928, 4)

(1928,)

(643, 4)

(643,)

(643, 4)

(643,)

## Исследование моделей

### Дерево решений ###

Исследуем дерево решений. Для получения оптимального результата рассмотрим модели с разной глубиной дерева от 2 до 5. Большую глубину брать не будем для исключения переобучения.

In [12]:
best_model_1 = None
best_depth_1 = 0
best_accuracy_1 = 0
for depth in range(2, 6):
    model_1 = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_1.fit(features_train, target_train)
    accuracy_1 = model_1.score(features_valid, target_valid)
    if accuracy_1 > best_accuracy_1:
        best_accuracy_1 = accuracy_1
        best_model_1 = model_1
        best_depth_1 = depth

In [13]:
print('Лучший результат дерева решений -', best_accuracy_1)
print('Лучшая глубина дерева решений -', best_depth_1)

Лучший результат дерева решений - 0.8164852255054432
Лучшая глубина дерева решений - 5


Дерево решений показывает достаточно высокое значение accuracy, при этом скорость обработки данных тоже высокая.

### Случайный лес ###

Теперь попробуем найти лучшую модель случайного леса. Переберем различные варианты количества деревьев в диапазоне от 10 до 300 с шагом 10. Глубину также зададим от 2 до 5.

In [14]:
best_model_2 = None
best_depth_2 = 0
best_est_2 = 0
best_accuracy_2 = 0
for est in range(10, 300, 10):
    for depth in range(2, 6):
        model_2 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, n_jobs=-1)
        model_2.fit(features_train, target_train)
        accuracy_2 = model_2.score(features_valid, target_valid)
        if accuracy_2 > best_accuracy_2:
            best_accuracy_2 = accuracy_2
            best_model_2 = model_2
            best_depth_2 = depth
            best_est_2 = est

In [15]:
print('Лучший результат случайного леса -', best_accuracy_2)
print('Лучшее количество деревьев случайного леса -', best_est_2)
print('Лучшая глубина случайного леса -', best_depth_2)

Лучший результат случайного леса - 0.8211508553654744
Лучшее количество деревьев случайного леса - 10
Лучшая глубина случайного леса - 4


Несмотря на низкую скорость работы, случайный лес позволяет достичь более высокого значения accuracy на валидационной выборке.

### Логистическая регрессия ###

Также построим модель, основанную на логистической регрессии.

In [16]:
model_3 = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_3.fit(features_train, target_train)
best_accuracy_3 = model_3.score(features_valid, target_valid)

In [17]:
print('Лучший результат логистической регрессии -', best_accuracy_3)

Лучший результат логистической регрессии - 0.7558320373250389


Несмотря на очень высокую скорость обработки данных, результат accuracy ниже чем у других моделей.

**Лучшее значение accuracy показала модель, построенная на основе случайного леса с 10 деревьями и глубиной равной 4. Проверим ее на тестовой выборке.**

## Проверка модели на тестовой выборке

Снова построим модель на основе случайного леса с заранее заданными гиперпараметрами: количество деревьев - 10, глубина - 4.

Для улучшения качества обучения модели будем использовать суммарно выборки train и valid.

In [18]:
model = RandomForestClassifier(random_state=12345, n_estimators=10, max_depth=4)
model.fit(features_new, target_new)

RandomForestClassifier(max_depth=4, n_estimators=10, random_state=12345)

Предсказания построенной модели:

In [19]:
predictions = model.predict(features_test)

Доля правильных ответов:

In [20]:
accuracy_score(target_test, predictions)

0.7978227060653188

Точность (доля правильно предсказанных пользователей с тарифом ultra):

In [21]:
precision_score(target_test, predictions)

0.8252427184466019

Полнота (доля пользователей с тарифом ultra, которых выделила модель):

In [22]:
recall_score(target_test, predictions)

0.43147208121827413

Гармоническое среднее значение запоминания и точности:

In [23]:
f1_score(target_test, predictions)

0.5666666666666667

Построение confusion_matrix:

In [24]:
confusion_matrix(target_test, predictions)

array([[428,  18],
       [112,  85]])

Согласно confusion_matrix получаем следующий результат:
- 428 правильно предсказанных тарифов smart
- 112 неправильно предсказанных тарифов smart
- 18 неправильно предсказанных тарифов ultra
- 85 правильно предсказанных тарифов ultra

Построенная модель может помочь бизнесу, предлагая пользователю те тарифы, которые выгодны компании. Тем самым можно сократить издержки на поддержание старых тарифов. Это также может быть полезно для самих пользователей, которые регулярно не укладываются в пакет услуг, предусмотренных используемым тарифом, или наоборот не расходуют его полностью.

## Проверка модели на адекватность

Для того, чтобы проверить целесообразность создания и использования модели машинного обучения, сравним ее с моделью, которая по умолчанию предлагает пользователю один из тарифов.

Построим модель, которая для всех пользователей предлагает наиболее часто встречающийся тариф, и сравним ее значение accuracy с ранее построенной моделью на основе случайного леса.

In [25]:
model_test = DummyClassifier(random_state=12345, strategy='most_frequent')
model_test.fit(features_train, target_train)
accuracy_test = model_test.score(features_test, target_test)

In [26]:
accuracy_test

0.6936236391912908

Значение accuracy получилось ниже и не соответствует целевому значению (0.75). Значит, использование построенной модели на основе случайного леса для рекомендации тарифа целесообразно.